In [3]:
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import numpy as np

In [14]:
# Первое задание
img: Image.Image = Image.open("test.png")
# получение минимального и максимального значения функции яркости изображения
b_min, b_max = get_brightness_min_and_max(img)
# получение гистограммы изображения
plot_hist(img, "originalFig.png")


<Figure size 432x288 with 0 Axes>

In [16]:
# Второе задание
b1_min, b1_max = (0, 255)
# растяжение изображения из одного диапазона значений в другой диапазон
img1 = stretch_image_range(img, (b_min, b_max), (b1_min, b1_max), show=False)
plot_hist(img1, "linearFIG.png", show=False)


<Figure size 432x288 with 0 Axes>

In [17]:
# Первое задание продолжение
# Поиск погорового значения на изорбажении с отмасшатабированым диапазоном функции яркости
threshold = find_threshold(img1)
# Масшабирование порогового значения в старый диапазон
threshold = range_convert(threshold, (b1_min, b1_max), (b_min, b_max))
    
img0 = threshold_proc(img, threshold, show=False)
plot_hist(img0, "thresholdFig.png", show=False)


<Figure size 432x288 with 0 Axes>

In [31]:
# Третье задание
# params[0] - диапазон, где лежит "порог для выреза"
# params[1] - коэффициенты c1,c2,c3, c1 и c3 - коэффициенты лнейной функции, c2 - константа для порога
params = [(213, 225), (0, 255, 0)]
img2 = elem_wise_transform(img, params, f1, show=False)
plot_hist(img2, "elemWiseFIG.png", show=False)


<Figure size 432x288 with 0 Axes>

In [5]:
def plot_hist(img, name, show=False):
    img = ImageOps.grayscale(img)

    hist = img.histogram()

    plt.ylim(bottom=-0.2, top=np.max(hist) + 15)
    plt.bar(range(0, 256), hist)
    plt.grid()
    if show:
        plt.show()
    plt.savefig(name, dpi=250)
    plt.clf()


In [6]:
def threshold_proc(img, threshold, show=False):
    img_tmp = ImageOps.grayscale(img)
    pix = img_tmp.load()
    for i in range(img.size[0]):
        for j in range(img.size[1]):
            if pix[i, j] < threshold:
                pix[i, j] = 0
            else:
                pix[i, j] = 255
    if show:
        img_tmp.show()
    img_tmp.save("threshold.png")
    return img_tmp


In [7]:
def get_brightness_min_and_max(img):
    img_arr = np.array(ImageOps.grayscale(img))
    return img_arr.min(), img_arr.max()


In [8]:
def range_convert(x, x_range, y_range):
    x_bot, x_top = x_range
    y_bot, y_top = y_range
    # сдвиг и масштабирование
    return (int)(((x - x_bot) / (x_top - x_bot)) * (y_top - y_bot) + y_bot)


In [9]:
def stretch_image_range(img, x_range, y_range, show=False):
    img_tmp = ImageOps.grayscale(img)
    pix = img_tmp.load()
    # проходимся по всем пикселям изображения и отображаем значения одного диапазона в другой
    for i in range(img.size[0]):
        for j in range(img.size[1]):
            pix[i, j] = range_convert(pix[i, j], x_range, y_range)
    if show:
        img_tmp.show()
    img_tmp.save("linear.png")
    return img_tmp


In [10]:
def clamp(x, a, b):
    if x > b:
        return int(b)
    elif x < a:
        return int(a)
    else:
        return int(x)


In [29]:
# функция преобразования одного значения
def f1(x, params):
    a, b = params[0]
    c1, c2, c3 = params[1]
    if x < a:
        return c1
    elif a <= x < b:
        return c2
    elif b <= x:
        return c3


In [12]:
#функция для поэлементного преобразования
def elem_wise_transform(img, params, f, show=False):
    img_tmp = ImageOps.grayscale(img)
    pix = img_tmp.load()
    #проходимся по пикселям изображения
    for i in range(img.size[0]):
        for j in range(img.size[1]):
            pix[i, j] = f(pix[i, j], params)
    if show:
        img_tmp.show()
    img_tmp.save("elemWise.png")
    return img_tmp


In [13]:
#функция для поиска порогового значения
def find_threshold(img):
    img = ImageOps.grayscale(img)
    hist = img.histogram()
    left, right = [np.where(hist == i)[0][0] for i in np.sort(hist)[-2:]]
    vals = hist[left:right]
    i = np.where(vals == np.min(np.take(vals, np.nonzero(vals))))
    return i[0][0] + left
